共通ライブラリ

In [6]:
from pandas.core.indexes.timedeltas import timedelta_range
import sys
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
import pandas as pd

日程表用

In [11]:
#日程表を1,2日目分けて入力
original_excel_path = "2023early_schedule.xlsx" #日程表のパス
sheet_names = ["一日目", '二日目']#メインの日程表のシート名
time_row = 7#開始時間が書かれている行-1を指定 (第8行→7)
place_column = 1#試合場所が書かれている列-1を指定 (B列→1)
season = 'Kouki'#Kouki or Zenki
credential_file_path = "rumutai-6b4ce-firebase-adminsdk-v4vb7-179084a91e.json"#credential_fileはfirestoreのサービスアカウント設定から取得できる。ぜったいに公開リポジトリに入れない。

column_names = ["id", "team1", "team2", "place", "hour", "minute"]

In [12]:
#データのよみこみ 
def ReadGamesFromExcel(row, col, original_df):
  if not pd.isna(original_df.iloc[row, col]):#nullチェック 
    id = original_df.iloc[row+3, col].lower() #小文字に変換する
    team1 = str(original_df.iloc[row+1, col])
    team2 = str(original_df.iloc[row+2, col])
    time = str(original_df.iloc[time_row, col])
    place = original_df.iloc[row, place_column]
    
    #時刻の分割
    hour_and_min = time.split(":")
    hour = hour_and_min[0]
    minute = hour_and_min[1]

    #DataFrameに変換 
    data = pd.DataFrame(data=[[id, team1, team2, place, hour, minute]], columns= column_names)
    return data

#整形
def SortedDataframe(original_df):
  aligned_df = pd.DataFrame(index = [], columns = column_names)
  
  for i in range(time_row + 1, len(original_df)):#行ごとに読み込む
    if (i - (time_row + 1)) % 4 == 0:#４行ずつ読み込む
      if pd.isna(original_df.iloc[i, place_column]): #下端に辿り着いたらループ解除
        break
      for j in range (place_column + 1, len(original_df.columns)):#列ごとに読み込み
        if pd.isna(original_df.iloc[time_row, j]):#右端に辿り着いたらループ解除
          break
        aligned_df = pd.concat([aligned_df,ReadGamesFromExcel(i, j, original_df)], ignore_index=True)

  sorted_df = aligned_df.sort_values("id")#idじゅんに並び替え
  return sorted_df

#Excelファイルの読み込み
multi_excel_sheets = pd.read_excel(original_excel_path, sheet_name=None)
df_sheets = []

#必要なシートを取り出す
for sheet_name in multi_excel_sheets.keys():
    if sheet_name in sheet_names:
      df = multi_excel_sheets[sheet_name] # sheet_nameのシートをDataFrameとしてdfへ
      df_sheets.append(df)

#試合じょうほうを読み取って整形する
sorted_sheets = []
for df in df_sheets:
   sorted_sheets.append(SortedDataframe(df))

sorted_sheets

/Users/pe/anaconda3/envs/pandas/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/pe/anaconda3/envs/pandas/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/pe/anaconda3/envs/pandas/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/pe/anaconda3/envs/pandas/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,id,team1,team2,place,hour,minute
0,1d-a01,103,104,運動場B,09,10
1,1d-a02,103,105,運動場B,13,20
2,1d-a03,103,109,運動場B,12,30
3,1d-a05,104,109,運動場B,11,40
4,1d-a06,105,109,運動場C,09,35
...,...,...,...,...,...,...
209,3k-a09,305,309,小体育館,08,45
210,3k-f01,A1,B2,鯱光館東,12,05
211,3k-f02,B1,A2,鯱光館東,12,30
212,3k-f03,(F1),(F2),鯱光館東,13,20


In [17]:

if not firebase_admin._apps:
  cred = credentials.Certificate('rumutai-6b4ce-firebase-adminsdk-v4vb7-179084a91e.json')
  firebase_admin.initialize_app(cred)
  
db = firestore.client()

if season == 'Zenki':
   collection_ref = db.collection("gameDataZenki")
elif season == 'Kouki':
   collection_ref = db.collection("Test1")
else:
   print("seasonの値が正しく指定されていません")
   exit()

initial_data_ref = db.collection("dataForInit").document("dataForInitDoc")
initial_data = initial_data_ref.get().to_dict()

def UploadGameData(date, df):
  for r in range(0,len(df)):
    #データの再取得と補充
      extracted_df = df.loc[r]
      id = extracted_df["id"]
      team = {"0":extracted_df["team1"], "1":extracted_df["team2"]}
      place = extracted_df["place"]
      startTime = {"date":date, "hour":extracted_df["hour"], "minute": extracted_df["minute"]}
      if "rumutaiStaff" in df.columns:
        rumutaiStaff = extracted_df["rumutaiStaff"]
      else:
        rumutaiStaff = ""
      referees = {"0":'', "1":'', "2":''}
      extraTime = ""
      gameStatus = "before"
      score=["", ""]
      scoreDetail = {"0":["",""], "1":["", ""], "2":["", ""]}

      #sportの判定
      sports = initial_data['sports']
      category = id.split("-")[0]
      if(category in sports.keys()):
         sport = sports[category]
      else:
         sport = ""
      
      #アップロード
      doc_ref = collection_ref.document(id)
      doc_ref.set({
          'gameId': id,
          'team':team,
          'place': place,
          'startTime': startTime,
          "referees": referees,
          "rumutaiStaff":rumutaiStaff,
          'sport': sport,
          "score":score,
          "scoreDetail":scoreDetail,
          "extraTime":extraTime,
          "gameStatus":gameStatus,
      }, merge = True)

for i in range(2):
   UploadGameData(i, sorted_sheets[i])

KeyboardInterrupt: 

CredentialFileを公開リポジトリに置かないこと！！！

審判表用

In [24]:
original_excel_path = "2023early_referees.xlsx" #日程表のパス
sheet_name = "一覧表"#メインの日程表のシート名
id_column = 3#試合コードが書かれている列-1を指定 (D列→3)
chief_referee_number_column = 44#試合場所が書かれている列-1を指定 (AS列→44)
season = 'Kouki'#Kouki or Zenki
credential_file_path = "rumutai-6b4ce-firebase-adminsdk-v4vb7-179084a91e.json"#credential_fileはfirestoreのサービスアカウント設定から取得できる。ぜったいに公開リポジトリに入れない。

column_names = ["id", "referee0", 'referee1', 'referee2']

In [25]:
#データのよみこみ 
def ReadRefereesFromExcel(row, original_df):
  id = original_df.iloc[row, id_column].lower()
  referee0 = original_df.iloc[row, chief_referee_number_column]
  referee1 = original_df.iloc[row, chief_referee_number_column+4]
  referee2 = original_df.iloc[row, chief_referee_number_column+8]

  #DataFrameに変換 
  data = pd.DataFrame(data=[[id, referee0, referee1, referee2]], columns= column_names)
  return data

#整形
def SortedDataframe(original_df):
  aligned_df = pd.DataFrame(index = [], columns = column_names)
  for i in range(time_row + 1, len(original_df)):#行ごとに読み込む
    if pd.isna(original_df.iloc[i, id_column]): #下端に辿り着いたらループ解除
      break
    aligned_df = pd.concat([aligned_df,ReadRefereesFromExcel(i, original_df)], ignore_index=True)

  sorted_df = aligned_df.sort_values("id")#id順に並び替え
  return sorted_df

#Excelファイルの読み込み
excel_sheet_df = pd.read_excel(original_excel_path, sheet_name=sheet_name)
sorted_sheet = SortedDataframe(excel_sheet_df)

sorted_sheet

,id,referee0,referee1,referee2
13,1d-a01,10111,20537,20538
14,1d-a02,10706,20116,30304
15,1d-a03,10218,10139,20308
16,1d-a04,10224,30630,20116
17,1d-a05,10116,10212,30401
...,...,...,...,...
130,3k-b06,20339,10312,NaN
191,3k-f01,10401,20723,10914
192,3k-f02,20727,20723,10312
193,3k-f03,10201,20205,10421


In [ ]:

if not firebase_admin._apps:
  cred = credentials.Certificate('rumutai-6b4ce-firebase-adminsdk-v4vb7-179084a91e.json')
  firebase_admin.initialize_app(cred)
  
db = firestore.client()

if season == 'Zenki':
   collection_ref = db.collection("gameDataZenki")
elif season == 'Kouki':
   collection_ref = db.collection("Test1")
else:
   print("seasonの値が正しく指定されていません")
   exit()


def UploadRefereeData(df):
  for r in range(0,len(df)):
    #データの再取得と補充
      extracted_df = df.loc[r]
      id = extracted_df["id"]
      referees = {'0': extracted_df["referee0"], '1': extracted_df["referee1"], '2': extracted_df["referee2"] }
      
      #アップロード
      doc_ref = collection_ref.document(id)
      doc_ref.set({
          'referees': referees
      }, merge = True)


UploadRefereeData(sorted_sheet)